## 随机梯度下降法

<img src="../img/6.png" width="500" align="left"/>

模拟退火思想：$\eta =\dfrac {t_{0}}{i_{iters}+t_{1}}$

<img src="../img/7.png" width="500" align="left"/>

### 实现随机梯度下降

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
m = 100000

x = np.random.normal(size=m)
X = x.reshape(-1,1)
y = 4.*x + 3. + np.random.normal(0,3,size=m)

In [3]:
# 计算梯度
def dJ_vector(W,X_b_i,y_i):
    return X_b_i.T.dot(X_b_i.dot(W) - y_i) * 2.

# 梯度下降
def sgd(init_w,X,y,t0=5,t1=50,n_iters=1e4,init_b=0.):
    
    # 退火算法计算学习率
    def learning_rate(t):
        return t0 / (t + t1)
    
    # 补1
    X_b = np.hstack([np.ones((len(X),1)),X])
    # 补b
    W = np.hstack([init_b,init_w])
    
    for i_iter in range(n_iters):
        rand_i=np.random.randint(len(X_b))
        grad=dJ_vector(W,X_b[rand_i],y[rand_i])
        W=W-learning_rate(i_iter)*grad
        
    return W

In [4]:
%%time

init_w = np.zeros(X.shape[1])
W_ = sgd(init_w,X,y,n_iters=len(X)//3)
print(W_[0])
print(W_[1:])

3.051732346078063
[4.00602278]
CPU times: user 217 ms, sys: 3.89 ms, total: 221 ms
Wall time: 224 ms


### 为解决SGD随机性无法全面的使所有样本参与计算，改进一下

In [5]:
# 计算梯度
def dJ_vector(W,X_b_i,y_i):
    return X_b_i.T.dot(X_b_i.dot(W) - y_i) * 2.

# 梯度下降
def sgd2(init_w,X,y,t0=5,t1=50,n_iters=5,init_b=0.):
    
    # 退火算法计算学习率
    def learning_rate(t):
        return t0 / (t + t1)
    
    # 补1
    X_b = np.hstack([np.ones((len(X),1)),X])
    # 补b
    W = np.hstack([init_b,init_w])
    
    # 此时n_iters 单纯代表循环次数，与m无关
    for i_iter in range(n_iters):
        m = len(X_b)
        # 我们将矩阵乱序，然后每次使用一组计算梯度
        indexs = np.random.permutation(m)
        X_b_new = X_b[indexs]
        y_new = y[indexs]
        for i in range(m):
            grad=dJ_vector(W,X_b_new[i],y_new[i])
            W=W-learning_rate(i_iter*m+i)*grad
            
    return W

In [6]:
%%time

init_w = np.zeros(X.shape[1])
W_ = sgd2(init_w,X,y,n_iters=2)
print(W_[0])
print(W_[1:])

3.011831485793552
[4.004134]
CPU times: user 851 ms, sys: 11.1 ms, total: 862 ms
Wall time: 884 ms


### 使用真实的数据测试我们的SGD

In [7]:
# 导入波士顿房产数据
from sklearn import datasets
from sklearn.model_selection import train_test_split

boston = datasets.load_boston()
X=boston.data
y=boston.target

X=X[y<50]
y=y[y<50]

In [8]:
# 分割数据集
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2 ,random_state=666)

In [9]:
# 数据归一化
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
standardScaler.fit(X_train)
X_train_standard = standardScaler.transform(X_train)
X_test_standard = standardScaler.transform(X_test)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [10]:
# 预测函数
def predict_normal(W,X_test):
        X_b = np.hstack([np.ones((len(X_test),1)),X_test])
        y_hat = X_b.dot(W)
        return y_hat

# 引入R^2进行评价
from sklearn.metrics import r2_score

In [11]:
# 正规方程解
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
%time lin_reg.fit(X_train,y_train)
r2_score(y_test,lin_reg.predict(X_test))

print("系数:",lin_reg.coef_)
print("截距:",lin_reg.intercept_)

CPU times: user 13.5 ms, sys: 2.52 ms, total: 16 ms
Wall time: 12.1 ms


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

0.8129794056212809

系数: [-1.20354261e-01  3.64423279e-02 -3.61493155e-02  5.12978140e-02
 -1.15775825e+01  3.42740062e+00 -2.32311760e-02 -1.19487594e+00
  2.60101728e-01 -1.40219119e-02 -8.35430488e-01  7.80472852e-03
 -3.80923751e-01]
截距: 34.117399723229845


In [12]:
# SGD 随机梯度下降
init_w = np.zeros(X_train_standard.shape[1])
%time W_ = sgd2(init_w,X_train_standard,y_train,n_iters=2)
r2_score(y_test,predict_normal(W_,X_test_standard))
print("系数:",W_[1:])
print("截距:",W_[0])

CPU times: user 12.6 ms, sys: 1.96 ms, total: 14.6 ms
Wall time: 7.22 ms


0.8058104209961152

系数: [-0.74278529  0.4202707  -0.85441506  0.52013874 -1.55041305  2.35260848
 -0.53811922 -2.53880223  0.51701515 -0.40875924 -1.74174289  1.01033266
 -2.9560288 ]
截距: 21.48293966262566


In [13]:
%time W_ = sgd2(init_w,X_train_standard,y_train,n_iters=50)
r2_score(y_test,predict_normal(W_,X_test_standard))

CPU times: user 197 ms, sys: 3.59 ms, total: 200 ms
Wall time: 104 ms


0.8127913496243248

### sklearn 中的SGD

In [14]:
from sklearn.linear_model import SGDRegressor

In [15]:
sgd_reg = SGDRegressor()
%time sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard,y_test)

CPU times: user 5.84 ms, sys: 2.02 ms, total: 7.87 ms
Wall time: 3.62 ms


/Users/zc/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

0.8030568596339946

In [16]:
# 增大 n_iter 
sgd_reg = SGDRegressor(n_iter=100)
%time sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard,y_test)

CPU times: user 13.2 ms, sys: 1.17 ms, total: 14.4 ms
Wall time: 5.56 ms


/Users/zc/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=100, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

0.8130791166551363